# IRTM Practical Assignment: Sentiment Analysis in Rock Music

### Data Preperation

In [124]:
# Importing the necessary libraries

import lyricsgenius as genius
import pandas as pd
import string 
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import numpy as np
import string
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn')
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import word_tokenize
import nltk.data
import time   

from requests.exceptions import Timeout # To avoid timeout errors during pull
nltk.download('vader_lexicon')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Dogan\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dogan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [163]:
# Dataset from https://www.kaggle.com/lukaszamora/history-of-rock-19502020
# A Dataset containing popular rock songs from 1950 to 2020
# Lyrics are not present, but will be retrieved from lyricsgenius later

df = pd.read_csv("history-of-rock-spotify - Copy.csv")
df = df.drop('index', 1)
df

,name,artist,release_date,length,popularity,danceability,acousticness,danceability.1,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,time_signature,valence
0,Smells Like Teen Spirit,Nirvana,1991,5.032000,74,0.502,0.000025,0.502,0.912,0.000173,1,0.1060,-4.556,0.0564,116.761,4,0.720
1,Stairway to Heaven - Remaster,Led Zeppelin,1971,8.047167,78,0.338,0.580000,0.338,0.340,0.003200,9,0.1160,-12.049,0.0339,82.433,4,0.197
2,Bohemian Rhapsody - Remastered 2011,Queen,1975,5.905333,74,0.392,0.288000,0.392,0.402,0.000000,0,0.2430,-9.961,0.0536,143.883,4,0.228
3,Imagine - Remastered 2010,John Lennon,1971,3.131100,77,0.547,0.907000,0.547,0.257,0.183000,0,0.0935,-12.358,0.0252,75.752,4,0.169
4,(I Can't Get No) Satisfaction - Mono Version,The Rolling Stones,1965,3.713550,77,0.723,0.038300,0.723,0.863,0.031700,2,0.1280,-7.890,0.0338,136.302,4,0.931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5479,I'm In Your Mind,King Gizzard & The Lizard Wizard,2014,3.559833,47,0.296,0.005910,0.296,0.776,0.801000,6,0.5970,-5.630,0.0597,93.481,4,0.406
5480,Cellophane,King Gizzard & The Lizard Wizard,2014,3.179750,44,0.432,0.002130,0.432,0.887,0.916000,7,0.1200,-6.175,0.1230,92.965,4,0.357
5481,Hot Water,King Gizzard & The Lizard Wizard,2014,3.396450,40,0.627,0.860000,0.627,0.609,0.890000,9,0.1160,-9.387,0.0332,86.861,4,0.734
5482,Vitamin C - 2004 Remastered Version,CAN,1972,3.567767,52,0.643,0.006690,0.643,0.644,0.673000,4,0.1620,-12.615,0.0462,117.225,4,0.853


In [164]:
# Song name preprocessing
# Some songs include the names like 'Remastered' or 'Live', which often hinders the retrieved text that are supposed to be lyrics
# If a song is remastered or live, usually referred as 'Song - Remastered 20XX' or 'Song - 20XX Remaster', so the aim is the remove if a song name has the suffix '-' (Some song names will be removed, but they will be retrieved with the lyricsgenius query anyway)

def remove_suffix(series):
    for i in series:
        if '-' in series:
            index = series.index('-')
            series = series[:index]
    return series
    

In [165]:
# Applying the song name preprocessing function to the dataframe
# Normally, this wouldn't be needed, but the geniuslyrics API finds the remastered versions but additional/unrelated text are retrieved with the lyrics

df['name']=df['name'].apply(remove_suffix)
df

,name,artist,release_date,length,popularity,danceability,acousticness,danceability.1,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,time_signature,valence
0,Smells Like Teen Spirit,Nirvana,1991,5.032000,74,0.502,0.000025,0.502,0.912,0.000173,1,0.1060,-4.556,0.0564,116.761,4,0.720
1,Stairway to Heaven,Led Zeppelin,1971,8.047167,78,0.338,0.580000,0.338,0.340,0.003200,9,0.1160,-12.049,0.0339,82.433,4,0.197
2,Bohemian Rhapsody,Queen,1975,5.905333,74,0.392,0.288000,0.392,0.402,0.000000,0,0.2430,-9.961,0.0536,143.883,4,0.228
3,Imagine,John Lennon,1971,3.131100,77,0.547,0.907000,0.547,0.257,0.183000,0,0.0935,-12.358,0.0252,75.752,4,0.169
4,(I Can't Get No) Satisfaction,The Rolling Stones,1965,3.713550,77,0.723,0.038300,0.723,0.863,0.031700,2,0.1280,-7.890,0.0338,136.302,4,0.931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5479,I'm In Your Mind,King Gizzard & The Lizard Wizard,2014,3.559833,47,0.296,0.005910,0.296,0.776,0.801000,6,0.5970,-5.630,0.0597,93.481,4,0.406
5480,Cellophane,King Gizzard & The Lizard Wizard,2014,3.179750,44,0.432,0.002130,0.432,0.887,0.916000,7,0.1200,-6.175,0.1230,92.965,4,0.357
5481,Hot Water,King Gizzard & The Lizard Wizard,2014,3.396450,40,0.627,0.860000,0.627,0.609,0.890000,9,0.1160,-9.387,0.0332,86.861,4,0.734
5482,Vitamin C,CAN,1972,3.567767,52,0.643,0.006690,0.643,0.644,0.673000,4,0.1620,-12.615,0.0462,117.225,4,0.853


In [91]:
# Access token or genius API

access_token = 'V2qPRzBIY6oGEeS19sN_jWofhy5TEyvon4I8WgXso3LIOZpdS8io7Sq83hbY-lpD'

In [93]:
# Function to retrieve song lyrics as list
# An empty string is returned if lyricsgenius was unable to find for the particular index
# Timeout and retry mechanic is added since there are usually connection losses during the loop

def get_lyrics(df, access_token):
    lyrics=[]
    g = genius.Genius(access_token)
    g.timeout = 15
    g.sleep = 5
    for index, row in df.iterrows():
        artist = row['artist']
        title = row['name']      
        retries = 0 
        while retries < 3:
            print(index)
            try:
                song = g.search_song(title, artist)
                if song is None:
                    lyrics.append('')    
                else:
                    lyrics.append(song.lyrics)
                break 
                
            except Timeout as e: 
                retries += 1
                continue
                
    return lyrics

In [94]:
# Retrieving the lyrics as list with the same indices as the dataframe before
# Empty lyrics for instrumentals or the songs that lyricsgenius was unable to find
# Took about 5-6 hours

lyrics = get_lyrics(df, access_token)

0
Searching for "Smells Like Teen Spirit" by Nirvana...
Done.
1
Searching for "Stairway to Heaven " by Led Zeppelin...
Done.
2
Searching for "Bohemian Rhapsody " by Queen...
Done.
3
Searching for "Imagine " by John Lennon...
Done.
4
Searching for "(I Can't Get No) Satisfaction " by The Rolling Stones...
Done.
5
Searching for "Hotel California " by Eagles...
Done.
6
Searching for "Enter Sandman" by Metallica...
Done.
7
Searching for "Whole Lotta Love " by Led Zeppelin...
Done.
8
Searching for "Comfortably Numb" by Pink Floyd...
Done.
9
Searching for "One" by U2...
Done.
10
Searching for "Born to Run" by Bruce Springsteen...
Done.
11
Searching for "Smoke on the Water" by Deep Purple...
Done.
12
Searching for "Hey Jude " by The Beatles...
Done.
13
Searching for "Paranoid " by Black Sabbath...
Done.
14
Searching for "Seven Nation Army" by The White Stripes...
Done.
15
Searching for "Free Bird" by Lynyrd Skynyrd...
Done.
16
Searching for "Back In Black" by AC/DC...
Done.
17
Searching for "S

Done.
138
Searching for "Me and Bobby McGee" by Janis Joplin...
Done.
139
Searching for "Who Are You" by The Who...
Done.
140
Searching for "Brown Sugar " by The Rolling Stones...
Done.
141
Searching for "Black" by Pearl Jam...
Done.
142
Searching for "Lust For Life" by Iggy Pop...
Done.
143
Searching for "Carry on Wayward Son" by Kansas...
Done.
144
Searching for "Rock And Roll All Nite" by KISS...
Done.
145
Searching for "American Pie" by Don McLean...
Done.
146
Searching for "Cocaine" by Eric Clapton...
Done.
147
Searching for "Runnin' with the Devil " by Van Halen...
Done.
148
Searching for "Honky Tonk Women " by The Rolling Stones...
Done.
149
Searching for "Clocks" by Coldplay...
Done.
150
Searching for "Best of You" by Foo Fighters...
Done.
151
Searching for "Ziggy Stardust " by David Bowie...
Done.
152
Searching for "By the Way" by Red Hot Chili Peppers...
Done.
153
Searching for "Paranoid Android" by Radiohead...
Done.
154
Searching for "Personal Jesus " by Depeche Mode...
Don

Done.
269
Searching for "Would?" by Alice In Chains...
Done.
270
Searching for "Heart" by Nirvana...
Done.
271
Searching for "Closer" by Nine Inch Nails...
Done.
272
Searching for "Sober" by TOOL...
Done.
273
Searching for "Magic Carpet Ride" by Steppenwolf...
Done.
274
Searching for "Rock You Like A Hurricane" by Scorpions...
Done.
275
Searching for "Last Nite" by The Strokes...
Done.
276
Searching for "Fat Bottomed Girls " by Queen...
Done.
277
Searching for "The Middle" by Jimmy Eat World...
Done.
278
Searching for "Heroes " by David Bowie...
Done.
279
Searching for "Dancing In the Dark" by Bruce Springsteen...
Done.
280
Searching for "Touch Me" by The Doors...
Done.
281
Searching for "Louie Louie" by The Kingsmen...
Done.
282
Searching for "Life in the Fast Lane " by Eagles...
Done.
283
Searching for "Come Out And Play (Keep ‘Em Separated)" by The Offspring...
Done.
284
Searching for "Moondance " by Van Morrison...
Done.
285
Searching for "Float On" by Modest Mouse...
Done.
286
Sea

Done.
404
Searching for "Yellow Ledbetter" by Pearl Jam...
Done.
405
Searching for "Where It's At" by Beck...
Done.
406
Searching for "Call Me" by Blondie...
Done.
407
Searching for "California Dreamin' " by The Mamas & The Papas...
Done.
408
Searching for "Bat Out of Hell" by Meat Loaf...
Done.
409
Searching for "Like a Prayer" by Madonna...
Done.
410
Searching for "The Unforgiven" by Metallica...
Done.
411
Searching for "Purple Rain" by Prince...
Done.
412
Searching for "Love In An Elevator " by Aerosmith...
Done.
413
Searching for "Blurry" by Puddle Of Mudd...
Done.
414
Searching for "The Sound of Silence " by Simon & Garfunkel...
Done.
415
Searching for "Miss You " by The Rolling Stones...
Done.
416
Searching for "Slither" by Velvet Revolver...
Done.
417
Searching for "Gimme All Your Lovin'" by ZZ Top...
Done.
418
Searching for "Cochise" by Audioslave...
Done.
419
Searching for "Bulls On Parade" by Rage Against The Machine...
Done.
420
Searching for "Long Train Runnin'" by The Doob

Done.
535
Searching for "Do It Again" by Steely Dan...
Done.
536
Searching for "Intergalactic " by Beastie Boys...
Done.
537
Searching for "Listen to the Music" by The Doobie Brothers...
Done.
538
Searching for "Nothing Compares 2 U" by Sinéad O'Connor...
Done.
539
Searching for "Paralyzer" by Finger Eleven...
Done.
540
Searching for "My My, Hey Hey (Out of the Blue) " by Neil Young...
Done.
541
Searching for "Like a Stone" by Audioslave...
Done.
542
Searching for "I Walk the Line" by Johnny Cash...
Done.
543
Searching for "The Pretender" by Foo Fighters...
Done.
544
Searching for "Long Road To Ruin" by Foo Fighters...
Done.
545
Searching for "(Sittin' On) the Dock of the Bay" by Otis Redding...
Done.
546
Searching for "Sheena Is a Punk Rocker " by Ramones...
Done.
547
Searching for "Get Off Of My Cloud " by The Rolling Stones...
Done.
548
Searching for "Shoot to Thrill" by AC/DC...
Done.
549
Searching for "Higher" by Creed...
Done.
550
Searching for "Inside the Fire" by Disturbed...
D

...

Done.
4913
Searching for "Forever Tuesday Morning" by The Mockers...
Done.
4914
Searching for "Veteran of the Psychic Wars" by Blue Öyster Cult...
Done.
4915
Searching for "Keep Talking" by Pink Floyd...
Done.
4916
Searching for "Bigger Than Us" by White Lies...
Done.
4917
Searching for "Disappear" by Hoobastank...
Done.
4918
Searching for "All Those Years Ago " by George Harrison...
Done.
4919
Searching for "Hands Down" by Dashboard Confessional...
Done.
4920
Searching for "Take It On The Run " by REO Speedwagon...
Done.
4921
Searching for "Another Perfect Day" by American Hi-Fi...
Done.
4922
Searching for "Dope Nose" by Weezer...
Done.
4923
Searching for "Life Won't Wait" by Ozzy Osbourne...
Done.
4924
Searching for "All That I've Got" by Various Artists...
Done.
4925
Searching for "Nothing's Gonna Change My Love For You" by Glenn Medeiros...
Done.
4926
Searching for "Show Me The Way" by Styx...
Done.
4927
Searching for "Don't Bring Me Down (Bonus Track)" by The Animals...
Done.
4928

Done.
5046
Searching for "Love Ain't For Keeping" by The Who...
Done.
5047
Searching for "Outlook For Thursday " by DD Smash...
Done.
5048
Searching for "Undead" by Hollywood Undead...
Done.
5049
Searching for "Heaven Is a Halfpipe (If I Die)" by OPM...
No results found for: 'Heaven Is a Halfpipe (If I Die) OPM'
5050
Searching for "The Ghost Song" by The Doors...
Done.
5051
Searching for "Lost!" by Coldplay...
Done.
5052
Searching for "Farm on the Freeway " by Jethro Tull...
Done.
5053
Searching for "The Untouchable One" by Tom Cochrane...
Done.
5054
Searching for "Why Can't I Be You?" by The Cure...
Done.
5055
Searching for "I've Got a Rock 'N' Roll Heart" by Eric Clapton...
Done.
5056
Searching for "Cold Beverage" by G. Love & Special Sauce...
Done.
5057
Searching for "Tighten Up" by The Black Keys...
Done.
5058
Searching for "Whatever" by Oasis...
Done.
5059
Searching for "Red White & Blue" by Lynyrd Skynyrd...
Done.
5060
Searching for "Swing, Swing" by The All-American Rejects...
D

Done.
5175
Searching for "Roll On" by The Living End...
Done.
5176
Searching for "Stay" by Hurts...
Done.
5177
Searching for "Fall Down" by Toad The Wet Sprocket...
Done.
5178
Searching for "Footstompin' Music" by Grand Funk Railroad...
Done.
5179
Searching for "Nobody Does It Better" by Various Artists...
Done.
5180
Searching for "It's Saturday" by Marcy Playground...
Done.
5181
Searching for "Lonely Train" by Black Stone Cherry...
Done.
5182
Searching for "Blue Day" by Mi-Sex...
Done.
5183
Searching for "Blue Flower" by Mazzy Star...
Done.
5184
Searching for "Shock The Monkey" by Peter Gabriel...
Done.
5185
Searching for "Wave Babies" by Honeymoon Suite...
Done.
5186
Searching for "Too Much, Too Young, Too Fast" by Airbourne...
Done.
5187
Searching for "Black Rain" by Soundgarden...
Done.
5188
Searching for "Chained To The Wheel" by The Black Sorrows...
Done.
5189
Searching for "Rise Today" by Alter Bridge...
Done.
5190
Searching for "Give Me Love (Give Me Peace On Earth)" by George 

Done.
5300
Searching for "Вечно молодой" by Smyslovye Gallyutsinatsii...
No results found for: 'Вечно молодой Smyslovye Gallyutsinatsii'
5301
Searching for "45" by Shinedown...
Done.
5302
Searching for "800°" by Ebba Grön...
Done.
5303
Searching for "Everything's On TV" by The Hellacopters...
Done.
5304
Searching for "Talkin' Bout a Revolution" by Tracy Chapman...
Done.
5305
Searching for "1969 " by The Stooges...
Done.
5306
Searching for "Disorder" by Slayer...
Done.
5307
Searching for "Bachelorette" by Björk...
Done.
5308
Searching for "Sell Myself" by Thumb...
Done.
5309
Searching for "When You Sleep" by My Bloody Valentine...
Done.
5310
Searching for "D.O.A." by Bloodrock...
Done.
5311
Searching for "Don't Say Goodbye" by The Exponents...
Done.
5312
Searching for "J.A.R. (Jason Andrew Relva)" by Angus...
No results found for: 'J.A.R. (Jason Andrew Relva) Angus'
5313
Searching for "Land Of A Thousand Dances" by Little Richard...
Done.
5314
Searching for "Before Too Long" by Paul Kel

Done.
5430
Searching for "A Passage To Bangkok" by Rush...
Done.
5431
Searching for "Born Under Punches (The Heat Goes On)" by Talking Heads...
Done.
5432
Searching for "Jump" by Van Halen...
Done.
5433
Searching for "Panama" by Van Halen...
Done.
5434
Searching for "Top Jimmy " by Van Halen...
Done.
5435
Searching for "Runnin' With The Devil" by Van Halen...
Done.
5436
Searching for "Blister in the Sun " by Violent Femmes...
Done.
5437
Searching for "Roundabout" by Yes...
Done.
5438
Searching for "Miserlou" by Dick Dale...
Specified song does not contain lyrics. Rejecting.
5439
Searching for "The Politics of Time" by Minutemen...
Done.
5440
Searching for "Planet Claire" by The B-52's...
Done.
5441
Searching for "The Metal East" by Lightning Bolt...
Done.
5442
Searching for "I Wanna Be Your Dog " by The Stooges...
Done.
5443
Searching for "Sharpie Smile" by Kamikaze Palm Tree...
Done.
5444
Searching for "Dominator" by SLIFT...
Done.
5445
Searching for "Pentagram on the Moon" by Earth T

In [166]:
# Saving the lyrics list as csv for further and faster use

d = {"lyrics":lyrics}
df_l = pd.DataFrame(d)
df_l.to_csv('lyrics.csv', sep='%')

### Loading the lyrics from the csv

In [167]:
df_lyrics = pd.read_csv('lyrics.csv', sep='%')
df_lyrics = df_lyrics.drop('Unnamed: 0', 1)
df_lyrics

,lyrics
0,"[Verse 1]\nLoad up on guns, bring your friends..."
1,[Intro]\n\n[Verse 1]\nThere's a lady who's sur...
2,[Intro]\nIs this the real life? Is this just f...
3,[Verse 1]\nImagine there's no heaven\nIt's eas...
4,[Instrumental Intro]\n\n[Chorus]\nI can't get ...
...,...
5479,[Verse 1]\nEverybody's lazy when they're tired...
5480,"[Refrain]\nDo do do do, do do do do do do do\n..."
5481,"[Refrain]\nWater, hot water\nHot water, hot wa..."
5482,[Verse 1]\nHer daddy got a big aeroplane\nHer ...


In [168]:
# Adding the lyrics to the original dataframe
df['lyrics'] = df_lyrics['lyrics']
df

,name,artist,release_date,length,popularity,danceability,acousticness,danceability.1,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,time_signature,valence,lyrics
0,Smells Like Teen Spirit,Nirvana,1991,5.032000,74,0.502,0.000025,0.502,0.912,0.000173,1,0.1060,-4.556,0.0564,116.761,4,0.720,"[Verse 1]\nLoad up on guns, bring your friends..."
1,Stairway to Heaven,Led Zeppelin,1971,8.047167,78,0.338,0.580000,0.338,0.340,0.003200,9,0.1160,-12.049,0.0339,82.433,4,0.197,[Intro]\n\n[Verse 1]\nThere's a lady who's sur...
2,Bohemian Rhapsody,Queen,1975,5.905333,74,0.392,0.288000,0.392,0.402,0.000000,0,0.2430,-9.961,0.0536,143.883,4,0.228,[Intro]\nIs this the real life? Is this just f...
3,Imagine,John Lennon,1971,3.131100,77,0.547,0.907000,0.547,0.257,0.183000,0,0.0935,-12.358,0.0252,75.752,4,0.169,[Verse 1]\nImagine there's no heaven\nIt's eas...
4,(I Can't Get No) Satisfaction,The Rolling Stones,1965,3.713550,77,0.723,0.038300,0.723,0.863,0.031700,2,0.1280,-7.890,0.0338,136.302,4,0.931,[Instrumental Intro]\n\n[Chorus]\nI can't get ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5479,I'm In Your Mind,King Gizzard & The Lizard Wizard,2014,3.559833,47,0.296,0.005910,0.296,0.776,0.801000,6,0.5970,-5.630,0.0597,93.481,4,0.406,[Verse 1]\nEverybody's lazy when they're tired...
5480,Cellophane,King Gizzard & The Lizard Wizard,2014,3.179750,44,0.432,0.002130,0.432,0.887,0.916000,7,0.1200,-6.175,0.1230,92.965,4,0.357,"[Refrain]\nDo do do do, do do do do do do do\n..."
5481,Hot Water,King Gizzard & The Lizard Wizard,2014,3.396450,40,0.627,0.860000,0.627,0.609,0.890000,9,0.1160,-9.387,0.0332,86.861,4,0.734,"[Refrain]\nWater, hot water\nHot water, hot wa..."
5482,Vitamin C,CAN,1972,3.567767,52,0.643,0.006690,0.643,0.644,0.673000,4,0.1620,-12.615,0.0462,117.225,4,0.853,[Verse 1]\nHer daddy got a big aeroplane\nHer ...


In [169]:
# Cleaning the songs in which lyrics were not captured
df_new = df.dropna(subset=['lyrics'])
#Last index song is manually removed due to inaccurate lyric retrieval
df_new.drop(df_new.tail(1).index,inplace=True)
print(str(len(df)-len(df_new))+' songs with no retrieved lyrics removed.')
df_new

112 songs with no retrieved lyrics removed.


C:\Users\Dogan\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,name,artist,release_date,length,popularity,danceability,acousticness,danceability.1,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,time_signature,valence,lyrics
0,Smells Like Teen Spirit,Nirvana,1991,5.032000,74,0.502,0.000025,0.502,0.912,0.000173,1,0.1060,-4.556,0.0564,116.761,4,0.720,"[Verse 1]\nLoad up on guns, bring your friends..."
1,Stairway to Heaven,Led Zeppelin,1971,8.047167,78,0.338,0.580000,0.338,0.340,0.003200,9,0.1160,-12.049,0.0339,82.433,4,0.197,[Intro]\n\n[Verse 1]\nThere's a lady who's sur...
2,Bohemian Rhapsody,Queen,1975,5.905333,74,0.392,0.288000,0.392,0.402,0.000000,0,0.2430,-9.961,0.0536,143.883,4,0.228,[Intro]\nIs this the real life? Is this just f...
3,Imagine,John Lennon,1971,3.131100,77,0.547,0.907000,0.547,0.257,0.183000,0,0.0935,-12.358,0.0252,75.752,4,0.169,[Verse 1]\nImagine there's no heaven\nIt's eas...
4,(I Can't Get No) Satisfaction,The Rolling Stones,1965,3.713550,77,0.723,0.038300,0.723,0.863,0.031700,2,0.1280,-7.890,0.0338,136.302,4,0.931,[Instrumental Intro]\n\n[Chorus]\nI can't get ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5478,The River,King Gizzard & The Lizard Wizard,2015,10.170883,40,0.485,0.532000,0.485,0.737,0.503000,6,0.1920,-4.553,0.0418,115.038,4,0.529,[Verse 1]\nOnce you're in the zone\nThe river ...
5479,I'm In Your Mind,King Gizzard & The Lizard Wizard,2014,3.559833,47,0.296,0.005910,0.296,0.776,0.801000,6,0.5970,-5.630,0.0597,93.481,4,0.406,[Verse 1]\nEverybody's lazy when they're tired...
5480,Cellophane,King Gizzard & The Lizard Wizard,2014,3.179750,44,0.432,0.002130,0.432,0.887,0.916000,7,0.1200,-6.175,0.1230,92.965,4,0.357,"[Refrain]\nDo do do do, do do do do do do do\n..."
5481,Hot Water,King Gizzard & The Lizard Wizard,2014,3.396450,40,0.627,0.860000,0.627,0.609,0.890000,9,0.1160,-9.387,0.0332,86.861,4,0.734,"[Refrain]\nWater, hot water\nHot water, hot wa..."


In [172]:
# Create decades of each song
# The extracted decades will be used for future visualizations and alsa will be considered as labels in one a sentiment/emotion-like analysis

def create_decades(df):
    df_new = df
    decades = []
    for year in df['release_date']:
        if 1950 <= year < 1960:
            decades.append("50s")       
        if 1960 <= year < 1970:
            decades.append("60s")        
        if 1970 <= year < 1980:
            decades.append("70s")
        if 1980 <= year < 1990:
            decades.append("80s")
        if 1990 <= year < 2000:
            decades.append("90s")
        if 2000 <= year < 2010:
            decades.append("00s")
        if 2010 <= year < 2020:
            decades.append("10s")
        if 2020 <= year:
            decades.append("20s")
    
    df_new['decade'] = decades
    return df_new

In [173]:
# Add decades to the dataframe

df_decades = create_decades(df_new)
df_decades

<ipython-input-172-3e2cc0f0c34a>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['decade'] = decades


,name,artist,release_date,length,popularity,danceability,acousticness,danceability.1,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,time_signature,valence,lyrics,decade
0,Smells Like Teen Spirit,Nirvana,1991,5.032000,74,0.502,0.000025,0.502,0.912,0.000173,1,0.1060,-4.556,0.0564,116.761,4,0.720,"[Verse 1]\nLoad up on guns, bring your friends...",90s
1,Stairway to Heaven,Led Zeppelin,1971,8.047167,78,0.338,0.580000,0.338,0.340,0.003200,9,0.1160,-12.049,0.0339,82.433,4,0.197,[Intro]\n\n[Verse 1]\nThere's a lady who's sur...,70s
2,Bohemian Rhapsody,Queen,1975,5.905333,74,0.392,0.288000,0.392,0.402,0.000000,0,0.2430,-9.961,0.0536,143.883,4,0.228,[Intro]\nIs this the real life? Is this just f...,70s
3,Imagine,John Lennon,1971,3.131100,77,0.547,0.907000,0.547,0.257,0.183000,0,0.0935,-12.358,0.0252,75.752,4,0.169,[Verse 1]\nImagine there's no heaven\nIt's eas...,70s
4,(I Can't Get No) Satisfaction,The Rolling Stones,1965,3.713550,77,0.723,0.038300,0.723,0.863,0.031700,2,0.1280,-7.890,0.0338,136.302,4,0.931,[Instrumental Intro]\n\n[Chorus]\nI can't get ...,60s
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5478,The River,King Gizzard & The Lizard Wizard,2015,10.170883,40,0.485,0.532000,0.485,0.737,0.503000,6,0.1920,-4.553,0.0418,115.038,4,0.529,[Verse 1]\nOnce you're in the zone\nThe river ...,10s
5479,I'm In Your Mind,King Gizzard & The Lizard Wizard,2014,3.559833,47,0.296,0.005910,0.296,0.776,0.801000,6,0.5970,-5.630,0.0597,93.481,4,0.406,[Verse 1]\nEverybody's lazy when they're tired...,10s
5480,Cellophane,King Gizzard & The Lizard Wizard,2014,3.179750,44,0.432,0.002130,0.432,0.887,0.916000,7,0.1200,-6.175,0.1230,92.965,4,0.357,"[Refrain]\nDo do do do, do do do do do do do\n...",10s
5481,Hot Water,King Gizzard & The Lizard Wizard,2014,3.396450,40,0.627,0.860000,0.627,0.609,0.890000,9,0.1160,-9.387,0.0332,86.861,4,0.734,"[Refrain]\nWater, hot water\nHot water, hot wa...",10s


In [175]:
# Function to clean lyrics before tokenization
def clean_lyrics(df):

    df = df_new
    df_new['cleaned lyrics'] = df_new['lyrics'].str.lower()
    
    # Remove next line character
    df_new['cleaned lyrics'] = df_new['cleaned lyrics'].str.replace("\n"," ")
    
    #In order to clean the lyrical annotations like '[Verse], [Chorus], ..'
    df_new['cleaned lyrics'] = df_new['cleaned lyrics'].str.replace(r'\[.*?\]'," ")
    
    #In order to clean echo lyrics usually represented in between regular parentheses
    df_new['cleaned lyrics'] = df_new['cleaned lyrics'].str.replace(r'\([^)]*\)'," ")
    
    return df_new

In [176]:
df_cleaned_lyrics = clean_lyrics(df_decades)
df_cleaned_lyrics

<ipython-input-175-a28e83c23513>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['cleaned lyrics'] = df_new['lyrics'].str.lower()
<ipython-input-175-a28e83c23513>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['cleaned lyrics'] = df_new['cleaned lyrics'].str.replace("\n"," ")
<ipython-input-175-a28e83c23513>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

,name,artist,release_date,length,popularity,danceability,acousticness,danceability.1,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,time_signature,valence,lyrics,decade,cleaned lyrics
0,Smells Like Teen Spirit,Nirvana,1991,5.032000,74,0.502,0.000025,0.502,0.912,0.000173,1,0.1060,-4.556,0.0564,116.761,4,0.720,"[Verse 1]\nLoad up on guns, bring your friends...",90s,"load up on guns, bring your friends it's fun..."
1,Stairway to Heaven,Led Zeppelin,1971,8.047167,78,0.338,0.580000,0.338,0.340,0.003200,9,0.1160,-12.049,0.0339,82.433,4,0.197,[Intro]\n\n[Verse 1]\nThere's a lady who's sur...,70s,there's a lady who's sure all that glitte...
2,Bohemian Rhapsody,Queen,1975,5.905333,74,0.392,0.288000,0.392,0.402,0.000000,0,0.2430,-9.961,0.0536,143.883,4,0.228,[Intro]\nIs this the real life? Is this just f...,70s,is this the real life? is this just fantasy?...
3,Imagine,John Lennon,1971,3.131100,77,0.547,0.907000,0.547,0.257,0.183000,0,0.0935,-12.358,0.0252,75.752,4,0.169,[Verse 1]\nImagine there's no heaven\nIt's eas...,70s,imagine there's no heaven it's easy if you t...
4,(I Can't Get No) Satisfaction,The Rolling Stones,1965,3.713550,77,0.723,0.038300,0.723,0.863,0.031700,2,0.1280,-7.890,0.0338,136.302,4,0.931,[Instrumental Intro]\n\n[Chorus]\nI can't get ...,60s,i can't get no satisfaction i can't get n...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5478,The River,King Gizzard & The Lizard Wizard,2015,10.170883,40,0.485,0.532000,0.485,0.737,0.503000,6,0.1920,-4.553,0.0418,115.038,4,0.529,[Verse 1]\nOnce you're in the zone\nThe river ...,10s,once you're in the zone the river flows down...
5479,I'm In Your Mind,King Gizzard & The Lizard Wizard,2014,3.559833,47,0.296,0.005910,0.296,0.776,0.801000,6,0.5970,-5.630,0.0597,93.481,4,0.406,[Verse 1]\nEverybody's lazy when they're tired...,10s,everybody's lazy when they're tired 'cause e...
5480,Cellophane,King Gizzard & The Lizard Wizard,2014,3.179750,44,0.432,0.002130,0.432,0.887,0.916000,7,0.1200,-6.175,0.1230,92.965,4,0.357,"[Refrain]\nDo do do do, do do do do do do do\n...",10s,"do do do do, do do do do do do do do do do d..."
5481,Hot Water,King Gizzard & The Lizard Wizard,2014,3.396450,40,0.627,0.860000,0.627,0.609,0.890000,9,0.1160,-9.387,0.0332,86.861,4,0.734,"[Refrain]\nWater, hot water\nHot water, hot wa...",10s,"water, hot water hot water, hot water hot wa..."


In [181]:
# This part includes VADER, but not as the core element of the sentiment analysis, but for the sake of creating sentiment labels to each song as positive or negative.
# I have looked for annotated lyrics data, but was able to find none (Something similar to IMDB review dataset)
# There isn't included any negation handling stuff here since this is for the labels and VADER just takes strings without the need of preprocessing

negative = []
neutral = []
positive = []
compound = []

#Initialize VADER
sid = SentimentIntensityAnalyzer()

#Iterate for each row of lyrics and append the scores
for index, row in df_cleaned_lyrics.iterrows():
    print(index)
    scores = sid.polarity_scores(row['cleaned lyrics'])
    negative.append(scores['neg'])
    neutral.append(scores['neu'])
    positive.append(scores['pos'])
    compound.append(scores['compound'])

#Create 4 columns to the main data frame  for each score 
df_cleaned_lyrics['negative'] = negative
df_cleaned_lyrics['neutral'] = neutral
df_cleaned_lyrics['positive'] = positive
df_cleaned_lyrics['compound'] = compound

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
27

1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2061
2062
2063
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2161
2162
2163
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200


3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736
3737
3738
3739
3740
3741
3742
3743
3744
3745
3746
3747
3748
3749
3750
3751
3752
3753
3755
3756
3757
3758
3759
3760
3761
3762
3763
3764
3765
3766
3767
3768
3769
3770
3771
3772
3773
3774
3775
3776
3777
3778
3779
3780
3781
3782
3783
3784
3785
3786
3787
3788
3789
3790
3791
3792
3793
3794
3795
3796
3797
3798
3799
3800
3801
3802
3803
3804
3805
3806
3807
3808
3809
3810
3811
3812
3813
3814
3815
3816
3817
3818
3819
3820
3821
3822
3823
3824
3825
3826
3827
3828
3829
3830
3831
3832
3833
3834
3835
3836
3838
3839
3840
3841
3842
3843
3844
3845
3846
3847
3848
3849
3850
3851
3852
3853
3854
3855
3856
3857
3858
3859
3860
3861
3862
3863
3864
3865
3866
3867
3868
3869
3870
3871
3872
3873
3874
3875
3876
3877
3878
3879
3880
3881
3882
3883
3884
3885
3886
3887
3888
3889
3890
3891
3892
3893
3894
3895
3896
3897
3898
3899
3900
3901
3902
3903
3904
3905
3906
3907
3908
3909
3910
3911
3912
3913
3914
3915
3916
3917
3918
3919
3920
3921
3922
3923
3924
3925


5407
5408
5409
5410
5411
5412
5413
5414
5415
5416
5417
5418
5419
5420
5421
5422
5423
5425
5426
5427
5428
5429
5430
5431
5432
5433
5434
5435
5436
5437
5439
5440
5441
5442
5443
5444
5445
5446
5447
5448
5449
5451
5452
5454
5455
5456
5457
5458
5459
5460
5462
5463
5464
5465
5466
5467
5468
5469
5470
5471
5472
5473
5474
5475
5476
5477
5478
5479
5480
5481
5482


<ipython-input-181-97c474e9ffb3>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned_lyrics['negative'] = negative
<ipython-input-181-97c474e9ffb3>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned_lyrics['neutral'] = neutral
<ipython-input-181-97c474e9ffb3>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [182]:
df_cleaned_lyrics

,name,artist,release_date,length,popularity,danceability,acousticness,danceability.1,energy,instrumentalness,...,tempo,time_signature,valence,lyrics,decade,cleaned lyrics,negative,neutral,positive,compound
0,Smells Like Teen Spirit,Nirvana,1991,5.032000,74,0.502,0.000025,0.502,0.912,0.000173,...,116.761,4,0.720,"[Verse 1]\nLoad up on guns, bring your friends...",90s,"load up on guns, bring your friends it's fun...",0.210,0.620,0.169,-0.7224
1,Stairway to Heaven,Led Zeppelin,1971,8.047167,78,0.338,0.580000,0.338,0.340,0.003200,...,82.433,4,0.197,[Intro]\n\n[Verse 1]\nThere's a lady who's sur...,70s,there's a lady who's sure all that glitte...,0.028,0.866,0.106,0.9771
2,Bohemian Rhapsody,Queen,1975,5.905333,74,0.392,0.288000,0.392,0.402,0.000000,...,143.883,4,0.228,[Intro]\nIs this the real life? Is this just f...,70s,is this the real life? is this just fantasy?...,0.202,0.690,0.108,-0.9840
3,Imagine,John Lennon,1971,3.131100,77,0.547,0.907000,0.547,0.257,0.183000,...,75.752,4,0.169,[Verse 1]\nImagine there's no heaven\nIt's eas...,70s,imagine there's no heaven it's easy if you t...,0.152,0.672,0.175,0.6775
4,(I Can't Get No) Satisfaction,The Rolling Stones,1965,3.713550,77,0.723,0.038300,0.723,0.863,0.031700,...,136.302,4,0.931,[Instrumental Intro]\n\n[Chorus]\nI can't get ...,60s,i can't get no satisfaction i can't get n...,0.106,0.655,0.238,0.9677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5478,The River,King Gizzard & The Lizard Wizard,2015,10.170883,40,0.485,0.532000,0.485,0.737,0.503000,...,115.038,4,0.529,[Verse 1]\nOnce you're in the zone\nThe river ...,10s,once you're in the zone the river flows down...,0.000,0.927,0.073,0.9477
5479,I'm In Your Mind,King Gizzard & The Lizard Wizard,2014,3.559833,47,0.296,0.005910,0.296,0.776,0.801000,...,93.481,4,0.406,[Verse 1]\nEverybody's lazy when they're tired...,10s,everybody's lazy when they're tired 'cause e...,0.130,0.870,0.000,-0.8689
5480,Cellophane,King Gizzard & The Lizard Wizard,2014,3.179750,44,0.432,0.002130,0.432,0.887,0.916000,...,92.965,4,0.357,"[Refrain]\nDo do do do, do do do do do do do\n...",10s,"do do do do, do do do do do do do do do do d...",0.031,0.969,0.000,-0.7268
5481,Hot Water,King Gizzard & The Lizard Wizard,2014,3.396450,40,0.627,0.860000,0.627,0.609,0.890000,...,86.861,4,0.734,"[Refrain]\nWater, hot water\nHot water, hot wa...",10s,"water, hot water hot water, hot water hot wa...",0.051,0.935,0.014,-0.7906


### Label Creation
For the sentiment and emotion analysis, the model to be formed will need annotated data.
One of the problems of trying to form this layers is the fact the features retrieved from the spotify dataset relate to audial aspect of the songs. Thus, the aim to predict categorized versions of these features from the lyrics may not give the best results in terms of performance. However, for the sentiment/emotion analysis, I will try to make 4 labels that I want to explore the results. The first label will be directly related to the lyrics, while the others possibly includes audial aspect in the labelling.

1) Lyrics Sentiment Labels formed by VADER: positive, neutral, negative

2) Decades Labels (To observe if there is a relation between the choice of the words and the decade the song is written in): 50s, 60s, 70s, 80s, 90s, 00s, 10s, 20s

3) Spotify feature valance as labels: happy, sad

4) Spotify feature energy as labels: up down

In [205]:
type(df_cleaned_lyrics['compound'][0])

numpy.float64

In [209]:
#This section will include 3 functions for creating 3 different labels

##Sentiment labels will be formed according to the compound scores retrieved by VADER
##The following thresholds will be used for labelling
###positive: compound score>=0.05
###neutral: compound score between -0.05 and 0.05
###negative: compound score<=-0.05

def sentiments_vader(df):
    df_new = df
    sentiments = []
    for score in df_new['compound']:
        if score>=0.05:
            sentiments.append("positive")   
        elif score<=-0.05:
            sentiments.append("negative")
        else:
            sentiments.append("neutral")
    df_new['Sentiment Labels'] = sentiments
    return df_new


def valence_label(df):
    df_new = df
    valence = []
    for score in df_new['valence']:
        if score>=0.5:
            valence.append("happy")
        else:
            valence.append("sad")           
    df_new['Valence Labels'] = valence
    return df_new

def energy_label(df):
    df_new = df
    energy = []
    for score in df_new['energy']:
        if score>=0.5:
            energy.append("up")
        else:
            energy.append("down")           
    df_new['Energy Labels'] = energy
    return df_new
    

In [211]:
df_final = sentiments_vader(df_cleaned_lyrics)
df_final = valence_label(df_final)
df_final = energy_label(df_final)
df_final

<ipython-input-209-e42dab3f1ae8>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['Sentiment Labels'] = sentiments
<ipython-input-209-e42dab3f1ae8>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['Valence Labels'] = valence
<ipython-input-209-e42dab3f1ae8>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

,name,artist,release_date,length,popularity,danceability,acousticness,danceability.1,energy,instrumentalness,...,lyrics,decade,cleaned lyrics,negative,neutral,positive,compound,Sentiment Labels,Valence Labels,Energy Labels
0,Smells Like Teen Spirit,Nirvana,1991,5.032000,74,0.502,0.000025,0.502,0.912,0.000173,...,"[Verse 1]\nLoad up on guns, bring your friends...",90s,"load up on guns, bring your friends it's fun...",0.210,0.620,0.169,-0.7224,negative,happy,up
1,Stairway to Heaven,Led Zeppelin,1971,8.047167,78,0.338,0.580000,0.338,0.340,0.003200,...,[Intro]\n\n[Verse 1]\nThere's a lady who's sur...,70s,there's a lady who's sure all that glitte...,0.028,0.866,0.106,0.9771,positive,sad,down
2,Bohemian Rhapsody,Queen,1975,5.905333,74,0.392,0.288000,0.392,0.402,0.000000,...,[Intro]\nIs this the real life? Is this just f...,70s,is this the real life? is this just fantasy?...,0.202,0.690,0.108,-0.9840,negative,sad,down
3,Imagine,John Lennon,1971,3.131100,77,0.547,0.907000,0.547,0.257,0.183000,...,[Verse 1]\nImagine there's no heaven\nIt's eas...,70s,imagine there's no heaven it's easy if you t...,0.152,0.672,0.175,0.6775,positive,sad,down
4,(I Can't Get No) Satisfaction,The Rolling Stones,1965,3.713550,77,0.723,0.038300,0.723,0.863,0.031700,...,[Instrumental Intro]\n\n[Chorus]\nI can't get ...,60s,i can't get no satisfaction i can't get n...,0.106,0.655,0.238,0.9677,positive,happy,up
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5478,The River,King Gizzard & The Lizard Wizard,2015,10.170883,40,0.485,0.532000,0.485,0.737,0.503000,...,[Verse 1]\nOnce you're in the zone\nThe river ...,10s,once you're in the zone the river flows down...,0.000,0.927,0.073,0.9477,positive,happy,up
5479,I'm In Your Mind,King Gizzard & The Lizard Wizard,2014,3.559833,47,0.296,0.005910,0.296,0.776,0.801000,...,[Verse 1]\nEverybody's lazy when they're tired...,10s,everybody's lazy when they're tired 'cause e...,0.130,0.870,0.000,-0.8689,negative,sad,up
5480,Cellophane,King Gizzard & The Lizard Wizard,2014,3.179750,44,0.432,0.002130,0.432,0.887,0.916000,...,"[Refrain]\nDo do do do, do do do do do do do\n...",10s,"do do do do, do do do do do do do do do do d...",0.031,0.969,0.000,-0.7268,negative,sad,up
5481,Hot Water,King Gizzard & The Lizard Wizard,2014,3.396450,40,0.627,0.860000,0.627,0.609,0.890000,...,"[Refrain]\nWater, hot water\nHot water, hot wa...",10s,"water, hot water hot water, hot water hot wa...",0.051,0.935,0.014,-0.7906,negative,happy,up


In [213]:
# Saving a slightly shorter lyrics data for next parts
df_data = pd.DataFrame()
df_data['Artist']=df_final['artist']
df_data['Title']=df_final['name']
df_data['Year']=df_final['release_date']
df_data['Decade']=df_final['decade']
df_data['Lyrics']=df_final['cleaned lyrics']
df_data['Compound Score']=df_final['compound']
df_data['Valence Score']=df_final['valence']
df_data['Energy Score']=df_final['energy']
df_data['Compound Label']=df_final['Sentiment Labels']
df_data['Valence Label']=df_final['Valence Labels']
df_data['Energy Label']=df_final['Energy Labels']
df_data

,Artist,Title,Year,Decade,Lyrics,Compound Score,Valence Score,Energy Score,Compound Label,Valence Label,Energy Label
0,Nirvana,Smells Like Teen Spirit,1991,90s,"load up on guns, bring your friends it's fun...",-0.7224,0.720,0.912,negative,happy,up
1,Led Zeppelin,Stairway to Heaven,1971,70s,there's a lady who's sure all that glitte...,0.9771,0.197,0.340,positive,sad,down
2,Queen,Bohemian Rhapsody,1975,70s,is this the real life? is this just fantasy?...,-0.9840,0.228,0.402,negative,sad,down
3,John Lennon,Imagine,1971,70s,imagine there's no heaven it's easy if you t...,0.6775,0.169,0.257,positive,sad,down
4,The Rolling Stones,(I Can't Get No) Satisfaction,1965,60s,i can't get no satisfaction i can't get n...,0.9677,0.931,0.863,positive,happy,up
...,...,...,...,...,...,...,...,...,...,...,...
5478,King Gizzard & The Lizard Wizard,The River,2015,10s,once you're in the zone the river flows down...,0.9477,0.529,0.737,positive,happy,up
5479,King Gizzard & The Lizard Wizard,I'm In Your Mind,2014,10s,everybody's lazy when they're tired 'cause e...,-0.8689,0.406,0.776,negative,sad,up
5480,King Gizzard & The Lizard Wizard,Cellophane,2014,10s,"do do do do, do do do do do do do do do do d...",-0.7268,0.357,0.887,negative,sad,up
5481,King Gizzard & The Lizard Wizard,Hot Water,2014,10s,"water, hot water hot water, hot water hot wa...",-0.7906,0.734,0.609,negative,happy,up


In [215]:
# Saving the datasets for the next step: sentiment/emotion analysis

df_final.to_csv('lyrics_all.csv', sep='%')
df_data.to_csv('lyrics_data.csv', sep='%')